## 1 - First Order Sequence Model
<hr>

Imagine that we want to develop a natural language computer interface, and we want to handle just three possible commands:

- Show me my directories please.
- Show me my files please.
- Show me my photos please.

Our vocabulary size is seven:

```
{directories, files, me, my, photos, please, show}
```

One useful way to represent sequences is with a transition model. For every word in the vocabulary, it shows what the next word is likely to be. If users ask about photos half the time, files 30% of the time, and directories the rest of the time, the transition model will look like this. The sum of the transitions away from any word will always add up to one.

<img src="images/markov_chain.png" width=600>

This particular transition model is called a **Markov chain,** because it satisfies the Markov property that the probabilities for the next word depend only on recent words. More specifically, it is a first order Markov model because it only looks at the single most recent word. If it considered the two most recent words it would be a second order Markov model.

It turns out that Markov chains can be expressed conveniently in matrix form. Each row represents one of the words in our vocabulary; so does each column. The matrix transition model treats a matrix as a lookup table. Find the row that corresponds to the word you're interested in. The value in each column shows the probability of that word coming next. Because the value of each element in the matrix represents a probability, they will all fall between zero and one. Because probabilities always sum to one, the values in each row will always add up to one.

<img src="images/transition_matrix.png" width=400>

If we just wanted to isolate the probabilities of which word comes after *my*, we can create a one-hot vector representing the word *my* and multiply it by our transition matrix. This pulls out the relevant row and shows us the probability distribution of what the next word will be.

<img src="images/transition_lookups.png" width=800>

## 2 - Second Order Sequence Model
<hr>

Predicting the next word based on only the current word is hard. That's like predicting the rest of a tune after being given just the first note. Our chances are a lot better if we can at least get two notes to go on.

We can see how this works in another toy language model for our computer commands. We expect that this one will only ever see two sentences, in a 40/60 proportion.

- Check whether the battery ran down please.
- Check whether the program ran please.

A Markov chain illustrates a first order model for this.

<img src="images/markov_chain_2.png" width=600>

Here we can see that if our model looked at the two most recent words, instead of just one, that it could do a better job. When it encounters *battery ran,* it knows that the next word will be *down,* and when it sees *program ran* the next word will be *please.* This eliminates one of the branches in the model, reducing uncertainty and increasing confidence. Looking back two words turns this into a **second order Markov model.** It gives more context on which to base next word predictions. Second order Markov chains are more challenging to draw, but here are the connections that demonstrate their value.

<img src="images/markov_chain_second_order.png" width=300>

To highlight the difference between the two, here is the first order transition matrix,

<img src="images/transition_matrix_first_order_2.png" width=500>

and here is the second order transition matrix.

<img src="images/transition_matrix_second_order.png" width=500>

Notice how the second order matrix has a separate row for every combination of words (most of which are not shown here). That means that if we start with a vocabulary size of $N$ then the transition matrix has $N^2$ rows.

What this buys us is more confidence. There are more ones and fewer fractions in the second order model. There's only one row with fractions in it, one branch in our model. Intuitively, looking at two words instead of just one gives more context, more information on which to base a next word guess.

### 2.1 - Second Order Sequence Model with Skips

A second order model works well when we only have to look back two words to decide what word comes next. What about when we have to look back further? Imagine we are building yet another language model. This one only has to represent two sentences, each equally likely to occur.

- Check the program log and find out whether it ran please.
- Check the battery log and find out whether it ran down please.

In this example, in order to determine which word should come after *ran,* we would have to look back 8 words into the past. If we want to improve on our second order language model, we can of course consider third- and higher order models. However, with a significant vocabulary size this takes a combination of creativity and brute force to execute. A naive implementation of an eighth order model would have $N^8$ rows, a ridiculous number for any reasonable vocubulary.

Instead, we can do something sly and make a second order model, but consider the combinations of the most recent word with each of the words that came before. It's still second order, because we're only considering two words at a time, but it allows us to reach back further and capture **long range dependencies.** The difference between this second-order-with-skips and a full umpteenth-order model is that we discard most of the word order information and combinations of preceeeding words. What remains is still pretty powerful.

Markov chains fail us entirely now, but we can still represent the link between each pair of preceding words and the words that follow. Here we've dispensed with numerical weights, and instead are showing only the arrows associated with non-zero weights. Larger weights are shown with heavier lines.

<img src="images/feature_voting.png" width=500>

Here's what it might look like in a transition matrix.

<img src="images/transition_matrix_second_order_skips.png" width=500>

This view only shows the rows relevant to predicting the word that comes after *ran.* It shows instances where the most recent word (ran) is preceded by each of the other words in the vocabulary. Only the relevant values are shown. All the empty cells are zeros.

## 3 - Masking
<hr>

We can sharpen the prediction by weeding out all the uninformative feature votes. With the exception of *battery, ran* and *program, ran.* It's helpful to remember at this point that we pull the relevant rows out of the transition matrix by multiplying it with a vector showing which features are currently active. For this example so far, we've been using the implied feature vector shown here.

<img src="images/feature_selection.png" width=500>

It includes a one for each feature that is a combination of ran with each of the words that come before it. Any words that come after it don't get included in the feature set. (In the next word prediction problem these haven't been seen yet, and so it's not fair to use them predict what comes next.) And this doesn't include all the other possible word combinations. We can safely ignore these for this example because they will all be zero.

To improve our results, we can additionally force the unhelpful features to zero by creating a **mask.** It's a vector full of ones except for the positions you'd like to hide or mask, and those are set to zero. In our case we'd like to mask everything except for *battery, ran* and *program, ran,* the only two features that have been of any help.

<img src="images/masked_feature_activities.png" width=500>

The mask has the effect of hiding a lot of the transition matrix. It hides the combination of *ran* with everything except *battery* and *program,* leaving just the features that matter.

<img src="images/masked_transition_matrix.png" width=500>

After masking the unhelpful features, the next word predictions become much stronger. When the word battery occurs earlier in the sentence, the word after ran is predicted to be down with a weight of 1 and please with a weight of 0. The same strong prediction occurs for please when program occurs early on.

This process of selective masking is the **attention** called out in the title of the original [paper](https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) on transformers.

### References
The contents of this notebook are based on the following article:

https://e2eml.school/transformers